In [174]:
import numpy as np # linear algebra
import pandas as pd
import plotly.express as px
import seaborn as sns
import matplotlib.pyplot as plt
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report,precision_score,recall_score,log_loss
from sklearn.model_selection import train_test_split,StratifiedKFold,GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder,OneHotEncoder,StandardScaler,MinMaxScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier,GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
import xgboost as xgb
import tensorflow
from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from imblearn.over_sampling import SMOTE
from collections import defaultdict
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression
from sklearn.impute import KNNImputer,SimpleImputer
from category_encoders import TargetEncoder
from sklearn.feature_selection import SelectFromModel
from lightgbm import LGBMClassifier

In [175]:
# import zipfile
# import os

# base_dir = '/kaggle/input/bnp-paribas-cardif-claims-management/train.csv.zip'

# with zipfile.ZipFile(base_dir,'r') as z:
#     z.extractall()


"\nimport zipfile\nimport os\n\nbase_dir = '/kaggle/input/bnp-paribas-cardif-claims-management/train.csv.zip'\n\nwith zipfile.ZipFile(base_dir,'r') as z:\n    z.extractall()\n"

In [176]:
train_data = pd.read_csv(r'C:\ML\datasets\bnp paribas\train.csv')
#train_data = pd.read_csv('./train.csv')

In [177]:
# train_data = train_data.iloc[:2000,:]
#train_data = train_data.iloc[:58000,:]

In [178]:
pd.set_option('display.max_columns',None)

In [179]:
train_data.head()

,ID,target,v1,v2,v3,v4,v5,v6,v7,v8,v9,v10,v11,v12,v13,v14,v15,v16,v17,v18,v19,v20,v21,v22,v23,v24,v25,v26,v27,v28,v29,v30,v31,v32,v33,v34,v35,v36,v37,v38,v39,v40,v41,v42,v43,v44,v45,v46,v47,v48,v49,v50,v51,v52,v53,v54,v55,v56,v57,v58,v59,v60,v61,v62,v63,v64,v65,v66,v67,v68,v69,v70,v71,v72,v73,v74,v75,v76,v77,v78,v79,v80,v81,v82,v83,v84,v85,v86,v87,v88,v89,v90,v91,v92,v93,v94,v95,v96,v97,v98,v99,v100,v101,v102,v103,v104,v105,v106,v107,v108,v109,v110,v111,v112,v113,v114,v115,v116,v117,v118,v119,v120,v121,v122,v123,v124,v125,v126,v127,v128,v129,v130,v131
0,3,1,1.335739,8.727474,C,3.921026,7.915266,2.599278,3.176895,0.012941,9.999999,0.503281,16.434108,6.085711,2.866830,11.636387,1.355013,8.571429,3.670350,0.106720,0.148883,18.869283,7.730923,XDX,-1.716131e-08,C,0.139412,1.720818,3.393503,0.590122,8.880867,C,A,1.083033,1.010829,7.270147,8.375452,11.326592,0.454546,0,4.012088,7.711453,7.653429,12.707581,2.015505,10.498338,9.848672,0.113561,C,12.171733,8.086643,0.899420,7.277792,G,16.747968,0.037096,1.299638,DI,3.971118,0.529802,10.890984,1.588448,15.858152,1,0.153461,6.363189,18.303925,C,9.314079,15.231789,17.142857,11.784549,F,1,1.614988,B,D,2.230940,7.292418,8.571429,E,3.000000,7.528326,8.861647,0.649820,1.299638,1.707317,0.866426,9.551836,3.321300,0.095678,0.905342,A,0.442252,5.814018,3.517720,0.462019,7.436824,5.454545,8.877414,1.191337,19.470199,8.389237,2.757375,4.374296,1.574039,0.007294,12.579184,E,2.382692,3.930922,B,0.433213,O,NaN,15.634907,2.857144,1.951220,6.592012,5.909091,-6.297423e-07,1.059603,0.803572,8.000000,1.989780,0.035754,AU,1.804126,3.113719,2.024285,0,0.636365,2.857144
1,4,1,NaN,NaN,C,NaN,9.191265,NaN,NaN,2.301630,NaN,1.312910,NaN,6.507647,NaN,11.636386,NaN,NaN,NaN,NaN,NaN,NaN,6.763110,GUV,NaN,C,3.056144,NaN,NaN,NaN,NaN,C,A,NaN,NaN,3.615077,NaN,14.579479,NaN,0,NaN,14.305766,NaN,NaN,NaN,NaN,NaN,2.449959,E,NaN,NaN,1.379210,NaN,G,NaN,1.129469,NaN,DY,NaN,NaN,NaN,NaN,NaN,2,2.544736,NaN,NaN,A,NaN,NaN,NaN,12.053353,F,2,NaN,B,D,NaN,NaN,NaN,D,NaN,7.277655,3.430691,NaN,NaN,NaN,NaN,9.848004,NaN,2.678584,NaN,B,NaN,NaN,NaN,NaN,NaN,NaN,8.303967,NaN,NaN,NaN,NaN,NaN,NaN,1.505335,NaN,B,1.825361,4.247858,A,NaN,U,G,10.308044,NaN,NaN,10.595357,NaN,NaN,NaN,NaN,NaN,NaN,0.598896,AF,NaN,NaN,1.957825,0,NaN,NaN
2,5,1,0.943877,5.310079,C,4.410969,5.326159,3.979592,3.928571,0.019645,12.666667,0.765864,14.756098,6.384670,2.505589,9.603542,1.984127,5.882353,3.170847,0.244541,0.144258,17.952332,5.245035,FQ,-2.785053e-07,E,0.113997,2.244897,5.306122,0.836005,7.499999,NaN,A,1.454082,1.734693,4.043864,7.959184,12.730517,0.259740,0,7.378964,13.077201,6.173469,12.346939,2.926830,8.897561,5.343819,0.126035,C,12.711328,6.836734,0.604504,9.637627,F,15.102041,0.085573,0.765305,AS,4.030613,4.277456,9.105481,2.151361,16.075602,1,0.123643,5.517949,16.377205,A,8.367347,11.040463,5.882353,8.460654,B,3,2.413618,B,B,1.963971,5.918368,11.764705,E,3.333334,10.194433,8.266200,1.530611,1.530613,2.429906,1.071429,8.447465,3.367346,0.111388,0.811447,G,0.271480,5.156559,4.214944,0.309657,5.663265,5.974026,11.588858,0.841837,15.491329,5.879353,3.292788,5.924457,1.668401,0.008275,11.670572,C,1.375753,1.184211,B,3.367348,S,NaN,11.205561,12.941177,3.129253,3.478911,6.233767,-2.792745e-07,2.138728,2.238806,9.333333,2.477596,0.013452,AE,1.773709,3.922193,1.120468,2,0.883118,1.176472
3,6,1,0.797415,8.304757,C,4.225930,11.627438,2.097700,1.987549,0.171947,8.965516,6.542669,16.347483,9.646653,3.903302,14.094723,1.945044,5.517242,3.610789,1.224114,0.231630,18.376407,7.517125,ACUE,-4.805344e-07,D,0.148843,1.308269,2.303640,8.926662,8.874521,C,B,1.587644,1.666667,8.703550,8.898468,11.302795,0.433735,0,0.287322,11.523045,7.931035,12.935823,1.470878,12.708574,9.670823,0.108387,C,12.194855,8.591954,3.329176,4.780357,H,16.621695,0.139721,1.178161,BW,3.965517,1.732102,11.777912,1.229246,15.927390,1,0.140260,6.292979,17.011645,A,9.703065,18.568129,9.425288,13.594728,F,2,2.272541,B,D,2.188198,8.213602,13.448277,B,1.947261,4.797873,13.315819,1.681034,1.379310,1.587045,1.242817,10.

In [180]:
train_data.shape

(2000, 133)

In [181]:
train_data.dtypes

ID          int64
target      int64
v1        float64
v2        float64
v3         object
           ...   
v127      float64
v128      float64
v129        int64
v130      float64
v131      float64
Length: 133, dtype: object

Missing data

In [182]:
train_data.isnull().sum()

ID          0
target      0
v1        863
v2        862
v3         65
         ... 
v127      863
v128      838
v129        0
v130      864
v131      864
Length: 133, dtype: int64

percentage of missing values in each feature

In [183]:
for feature in train_data.columns:
    no_of_missing_records = train_data[feature].isnull().sum()
    missing_percentage = ((no_of_missing_records)/train_data.shape[0])*100
    if missing_percentage>2:
        print("Missing percentage",feature ," ",missing_percentage)

Missing percentage v1   43.15
Missing percentage v2   43.1
Missing percentage v3   3.25
Missing percentage v4   43.1
Missing percentage v5   41.9
Missing percentage v6   43.15
Missing percentage v7   43.15
Missing percentage v8   41.9
Missing percentage v9   43.2
Missing percentage v11   43.15
Missing percentage v13   43.15
Missing percentage v15   43.15
Missing percentage v16   43.2
Missing percentage v17   43.1
Missing percentage v18   43.15
Missing percentage v19   43.2
Missing percentage v20   43.15
Missing percentage v23   43.65
Missing percentage v25   41.9
Missing percentage v26   43.15
Missing percentage v27   43.15
Missing percentage v28   43.15
Missing percentage v29   43.15
Missing percentage v30   52.55
Missing percentage v31   3.25
Missing percentage v32   43.15
Missing percentage v33   43.15
Missing percentage v35   43.15
Missing percentage v36   41.9
Missing percentage v37   43.2
Missing percentage v39   43.15
Missing percentage v41   43.15
Missing percentage v42   43.15

there are many features with missing values with across 40 percentage of data is missing.

In [184]:
train_data.drop(columns="ID",inplace=True)

In [185]:
train_data.describe()

,target,v1,v2,v4,v5,v6,v7,v8,v9,v10,v11,v12,v13,v14,v15,v16,v17,v18,v19,v20,v21,v23,v25,v26,v27,v28,v29,v32,v33,v34,v35,v36,v37,v38,v39,v40,v41,v42,v43,v44,v45,v46,v48,v49,v50,v51,v53,v54,v55,v57,v58,v59,v60,v61,v62,v63,v64,v65,v67,v68,v69,v70,v72,v73,v76,v77,v78,v80,v81,v82,v83,v84,v85,v86,v87,v88,v89,v90,v92,v93,v94,v95,v96,v97,v98,v99,v100,v101,v102,v103,v104,v105,v106,v108,v109,v111,v114,v115,v116,v117,v118,v119,v120,v121,v122,v123,v124,v126,v127,v128,v129,v130,v131
count,2000.000000,1.137000e+03,1138.000000,1138.000000,1162.000000,1137.000000,1.137000e+03,1162.000000,1136.000000,2.000000e+03,1137.000000,2000.000000,1.137000e+03,2000.000000,1.137000e+03,1.136000e+03,1138.000000,1137.000000,1136.000000,1137.000000,1986.000000,1.127000e+03,1162.000000,1.137000e+03,1137.000000,1137.000000,1137.000000,1.137000e+03,1.137000e+03,2000.000000,1137.000000,1.162000e+03,1.136000e+03,2000.000000,1.137000e+03,2000.000000,1137.000000,1137.000000,1.137000e+03,1138.000000,1.137000e+03,1162.000000,1138.000000,1137.000000,2000.000000,1127.000000,1137.000000,1162.000000,1.137000e+03,1137.000000,1.137000e+03,1138.000000,1.137000e+03,1138.000000,2000.000000,1162.000000,1138.000000,1137.000000,1137.000000,1137.000000,1.136000e+03,1162.000000,2000.00000,1.137000e+03,1138.000000,1137.000000,1.136000e+03,1.136000e+03,1162.000000,1162.000000,1.137000e+03,1.137000e+03,1.127000e+03,1.137000e+03,1162.000000,1.137000e+03,1162.000000,1137.000000,1136.000000,1137.000000,1137.000000,1136.000000,1137.000000,1.136000e+03,1161.000000,1.137000e+03,1.137000e+03,1138.000000,1118.000000,1.137000e+03,1.137000e+03,1162.000000,1138.000000,1162.000000,1.162000e+03,1.137000e+03,2000.000000,1.136000e+03,1.137000e+03,1.162000e+03,1136.000000,1.127000e+03,1.137000e+03,1.137000e+03,1136.000000,1127.000000,1.162000e+03,1137.000000,1.137000e+03,1162.000000,2000.000000,1136.000000,1.136000e+03
mean,0.761000,1.608348e+00,7.532984,4.131482,8.736983,2.426996,2.464934e+00,1.563782,9.040726,1.901050e+00,15.505263,6.888703,3.797855e+00,12.132218,2.054874e+00,4.944958e+00,3.852068,0.846339,0.223473,17.789241,7.058446,1.053463e+00,1.787235,1.850948e+00,2.732563,5.099007,8.238052,1.610122e+00,2.146660e+00,6.465515,8.165385,1.334127e+01,7.357589e-01,0.092500,1.275282e+00,10.390493,7.227496,12.928065,2.180269e+00,10.827303,9.155095e+00,1.700458,12.513625,8.060102,1.520570,7.165805,15.775119,1.318743,1.539145e+00,4.067249,7.835186e+00,10.601811,1.691320e+00,14.598802,1.045000,1.769514,6.367606,15.888853,9.311029,17.538069,9.410157e+00,12.314369,1.45550,2.409703e+00,2.420245,7.336480,1.335302e+01,2.219043e+00,7.265007,6.343210,2.158884e+00,1.601118e+00,2.803241e+00,1.210241e+00,10.194075,1.930491e+00,1.560632,0.961127,0.589508,5.502359,3.830750,0.672388,6.507932,7.611151e+00,7.598425,1.241220e+00,1.196032e+01,6.912623,2.950246,5.259080e+00,2.619964e+00,1.126228,11.816359,2.151992,4.191143e+00,3.359923e+00,13.645870,1.058386e+01,2.250587e+00,8.213556e+00,8.347814,3.179673e+00,1.282224e+00,2.701484e+00,6.824361,3.539575,9.846712e-01,1.666193,3.232882e+00,2.029463,0.318000,1.917826,1.704480e+00
std,0.426579,1.029360e+00,3.115834,1.133044,2.007807,0.591288,5.718982e-01,2.849095,1.784156,1.422560e+00,0.754558,0.935881,1.231579e+00,1.422021,7.171870e-01,1.863424e+00,1.906199,0.585155,0.132732,1.101663,1.095209,4.025272e+00,3.004921,5.159337e-01,0.836015,2.655900,1.255950,5.526744e-01,9.283647e-01,2.004399,1.341477,2.357879e+00,5.036742e-01,0.571058,2.485977e+00,3.145482,0.993040,0.996271,6.000365e-01,2.055597,2.016022e+00,2.955760,2.152208,0.896917,1.159206,2.408232,0.754427,2.335213,7.748303e-01,0.652934,6.833758e+00,2.007864,5.044844e-01,2.069609,0.731603,3.039902,2.505961,1.783952,1.113203,2.367664,1.966670e+00,2.318259,0.94099,7.867777e-01,1.380338,1.223347,1.919917e+00,1.046251e+00,2.119114,3.746519,1.003144e+00,8.985015e-01,1.359727e+00,4.586409e-01,2.976461,1.077981e+00,2.772992,0.156782,0.230002,1.654103,0.864122,0.254870,1.093271,1.981358e+00,2.323828,4.375180e-01,6.886204e+00,2.3784

it looks like there are negative values in all the features such as v1,v2

target distribution

In [186]:
class_balance = train_data["target"].value_counts().reset_index()

In [187]:
#fig = px.bar(class_balance,x="target",y="count")
#fig.show()

there is clearly target imbalance here 

In [188]:
y=train_data["target"]
#train_data.drop(columns='target',inplace=True)

In [189]:
integer_features = list(train_data.select_dtypes(include=['int64','float64']).columns)
categorical_features = list(train_data.select_dtypes(include='object').columns)
#print(integer_features)
#print(categorical_features)

In [190]:
integer_features = integer_features[1:]

In [191]:
#integer_features

to check if there are any correlated features

In [192]:
data_corr = train_data[integer_features].corr()

In [193]:
#data_corr

In [194]:
threshold = 0.5
corr_list = []
for i in range(len(integer_features)):
    for j in range(i+1,len(integer_features)):
        if (data_corr.iloc[i,j]>=0.5 and data_corr.iloc[i,j] <1) or (data_corr.iloc[i,j]<0 and data_corr.iloc[i,j]<-threshold):
            corr_list.append([data_corr.iloc[i,j],i,j])
corr_list = sorted(corr_list,key=lambda x : x[0],reverse=True)
for c,f1,f2 in corr_list:
    print(integer_features[f1] ,"and",integer_features[f2]," ",c)

it looks like there are multiple correlated features v17 and v76 .. . which means we can apply PCA

In [195]:
train_data.head()

,target,v1,v2,v3,v4,v5,v6,v7,v8,v9,v10,v11,v12,v13,v14,v15,v16,v17,v18,v19,v20,v21,v22,v23,v24,v25,v26,v27,v28,v29,v30,v31,v32,v33,v34,v35,v36,v37,v38,v39,v40,v41,v42,v43,v44,v45,v46,v47,v48,v49,v50,v51,v52,v53,v54,v55,v56,v57,v58,v59,v60,v61,v62,v63,v64,v65,v66,v67,v68,v69,v70,v71,v72,v73,v74,v75,v76,v77,v78,v79,v80,v81,v82,v83,v84,v85,v86,v87,v88,v89,v90,v91,v92,v93,v94,v95,v96,v97,v98,v99,v100,v101,v102,v103,v104,v105,v106,v107,v108,v109,v110,v111,v112,v113,v114,v115,v116,v117,v118,v119,v120,v121,v122,v123,v124,v125,v126,v127,v128,v129,v130,v131
0,1,1.335739,8.727474,C,3.921026,7.915266,2.599278,3.176895,0.012941,9.999999,0.503281,16.434108,6.085711,2.866830,11.636387,1.355013,8.571429,3.670350,0.106720,0.148883,18.869283,7.730923,XDX,-1.716131e-08,C,0.139412,1.720818,3.393503,0.590122,8.880867,C,A,1.083033,1.010829,7.270147,8.375452,11.326592,0.454546,0,4.012088,7.711453,7.653429,12.707581,2.015505,10.498338,9.848672,0.113561,C,12.171733,8.086643,0.899420,7.277792,G,16.747968,0.037096,1.299638,DI,3.971118,0.529802,10.890984,1.588448,15.858152,1,0.153461,6.363189,18.303925,C,9.314079,15.231789,17.142857,11.784549,F,1,1.614988,B,D,2.230940,7.292418,8.571429,E,3.000000,7.528326,8.861647,0.649820,1.299638,1.707317,0.866426,9.551836,3.321300,0.095678,0.905342,A,0.442252,5.814018,3.517720,0.462019,7.436824,5.454545,8.877414,1.191337,19.470199,8.389237,2.757375,4.374296,1.574039,0.007294,12.579184,E,2.382692,3.930922,B,0.433213,O,NaN,15.634907,2.857144,1.951220,6.592012,5.909091,-6.297423e-07,1.059603,0.803572,8.000000,1.989780,0.035754,AU,1.804126,3.113719,2.024285,0,0.636365,2.857144
1,1,NaN,NaN,C,NaN,9.191265,NaN,NaN,2.301630,NaN,1.312910,NaN,6.507647,NaN,11.636386,NaN,NaN,NaN,NaN,NaN,NaN,6.763110,GUV,NaN,C,3.056144,NaN,NaN,NaN,NaN,C,A,NaN,NaN,3.615077,NaN,14.579479,NaN,0,NaN,14.305766,NaN,NaN,NaN,NaN,NaN,2.449959,E,NaN,NaN,1.379210,NaN,G,NaN,1.129469,NaN,DY,NaN,NaN,NaN,NaN,NaN,2,2.544736,NaN,NaN,A,NaN,NaN,NaN,12.053353,F,2,NaN,B,D,NaN,NaN,NaN,D,NaN,7.277655,3.430691,NaN,NaN,NaN,NaN,9.848004,NaN,2.678584,NaN,B,NaN,NaN,NaN,NaN,NaN,NaN,8.303967,NaN,NaN,NaN,NaN,NaN,NaN,1.505335,NaN,B,1.825361,4.247858,A,NaN,U,G,10.308044,NaN,NaN,10.595357,NaN,NaN,NaN,NaN,NaN,NaN,0.598896,AF,NaN,NaN,1.957825,0,NaN,NaN
2,1,0.943877,5.310079,C,4.410969,5.326159,3.979592,3.928571,0.019645,12.666667,0.765864,14.756098,6.384670,2.505589,9.603542,1.984127,5.882353,3.170847,0.244541,0.144258,17.952332,5.245035,FQ,-2.785053e-07,E,0.113997,2.244897,5.306122,0.836005,7.499999,NaN,A,1.454082,1.734693,4.043864,7.959184,12.730517,0.259740,0,7.378964,13.077201,6.173469,12.346939,2.926830,8.897561,5.343819,0.126035,C,12.711328,6.836734,0.604504,9.637627,F,15.102041,0.085573,0.765305,AS,4.030613,4.277456,9.105481,2.151361,16.075602,1,0.123643,5.517949,16.377205,A,8.367347,11.040463,5.882353,8.460654,B,3,2.413618,B,B,1.963971,5.918368,11.764705,E,3.333334,10.194433,8.266200,1.530611,1.530613,2.429906,1.071429,8.447465,3.367346,0.111388,0.811447,G,0.271480,5.156559,4.214944,0.309657,5.663265,5.974026,11.588858,0.841837,15.491329,5.879353,3.292788,5.924457,1.668401,0.008275,11.670572,C,1.375753,1.184211,B,3.367348,S,NaN,11.205561,12.941177,3.129253,3.478911,6.233767,-2.792745e-07,2.138728,2.238806,9.333333,2.477596,0.013452,AE,1.773709,3.922193,1.120468,2,0.883118,1.176472
3,1,0.797415,8.304757,C,4.225930,11.627438,2.097700,1.987549,0.171947,8.965516,6.542669,16.347483,9.646653,3.903302,14.094723,1.945044,5.517242,3.610789,1.224114,0.231630,18.376407,7.517125,ACUE,-4.805344e-07,D,0.148843,1.308269,2.303640,8.926662,8.874521,C,B,1.587644,1.666667,8.703550,8.898468,11.302795,0.433735,0,0.287322,11.523045,7.931035,12.935823,1.470878,12.708574,9.670823,0.108387,C,12.194855,8.591954,3.329176,4.780357,H,16.621695,0.139721,1.178161,BW,3.965517,1.732102,11.777912,1.229246,15.927390,1,0.140260,6.292979,17.011645,A,9.703065,18.568129,9.425288,13.594728,F,2,2.272541,B,D,2.188198,8.213602,13.448277,B,1.947261,4.797873,13.315819,1.681034,1.379310,1.587045,1.242817,10.747144,1.40

In [196]:
for feature in integer_features:
    print(feature)
    sns.distplot(train_data[feature])
    plt.show()

'\nfor feature in integer_features:\n    print(feature)\n    sns.distplot(train_data[feature])\n    plt.show()\n'

In [197]:
#some features are not normally distributed
#some features are skewed
#some features are normally distributed

In [198]:
#train_data1 = train_data.copy()
#train_data1["target"] = y

In [199]:
#train_data1.head()
catboost_training_data = train_data.copy()

In [200]:
#sns.pairplot(train_data1[integer_features])

In [201]:
imputer = KNNImputer(n_neighbors=5)
train_data[integer_features] = imputer.fit_transform(train_data[integer_features])
categorical_imputer=SimpleImputer(strategy='most_frequent')
train_data[categorical_features]=categorical_imputer.fit_transform(train_data[categorical_features])

In [202]:
train_data[categorical_features].head()

,v3,v22,v24,v30,v31,v47,v52,v56,v66,v71,v74,v75,v79,v91,v107,v110,v112,v113,v125
0,C,XDX,C,C,A,C,G,DI,C,F,B,D,E,A,E,B,O,G,AU
1,C,GUV,C,C,A,E,G,DY,A,F,B,D,D,B,B,A,U,G,AF
2,C,FQ,E,C,A,C,F,AS,A,B,B,B,E,G,C,B,S,G,AE
3,C,ACUE,D,C,B,C,H,BW,A,F,B,D,B,B,B,B,J,G,CJ
4,C,HIT,E,C,A,I,H,BW,C,F,B,D,C,G,C,A,T,G,Z


In [203]:
for feature in train_data.columns:
    if train_data[feature].isnull().sum()>0:
        print(feature)

In [204]:
def encoding_categorical_data(categorical_features,train_data,target):
    #trying target encoding,frequency encoding
    #if len(train_data[feature].unique())>100:
    #df['categorical_feature_encoded'] = df['categorical_feature'].map(df['categorical_feature'].value_counts() / len(df))
    encoder = TargetEncoder(cols=categorical_features)
    train_data[categorical_features] = encoder.fit_transform(train_data[categorical_features],train_data[target])
    return train_data
        

In [205]:
#find out if there are any outliers in the features with box plot
# apply pca

In [206]:
train_data.head(10)

,target,v1,v2,v3,v4,v5,v6,v7,v8,v9,v10,v11,v12,v13,v14,v15,v16,v17,v18,v19,v20,v21,v22,v23,v24,v25,v26,v27,v28,v29,v30,v31,v32,v33,v34,v35,v36,v37,v38,v39,v40,v41,v42,v43,v44,v45,v46,v47,v48,v49,v50,v51,v52,v53,v54,v55,v56,v57,v58,v59,v60,v61,v62,v63,v64,v65,v66,v67,v68,v69,v70,v71,v72,v73,v74,v75,v76,v77,v78,v79,v80,v81,v82,v83,v84,v85,v86,v87,v88,v89,v90,v91,v92,v93,v94,v95,v96,v97,v98,v99,v100,v101,v102,v103,v104,v105,v106,v107,v108,v109,v110,v111,v112,v113,v114,v115,v116,v117,v118,v119,v120,v121,v122,v123,v124,v125,v126,v127,v128,v129,v130,v131
0,1,1.335739,8.727474,C,3.921026,7.915266,2.599278,3.176895,0.012941,9.999999,0.503281,16.434108,6.085711,2.866830,11.636387,1.355013,8.571429,3.670350,0.106720,0.148883,18.869283,7.730923,XDX,-1.716131e-08,C,0.139412,1.720818,3.393503,0.590122,8.880867,C,A,1.083033,1.010829,7.270147,8.375452,11.326592,0.454546,0.0,4.012088,7.711453,7.653429,12.707581,2.015505,10.498338,9.848672,0.113561,C,12.171733,8.086643,0.899420,7.277792,G,16.747968,0.037096,1.299638,DI,3.971118,0.529802,10.890984,1.588448,15.858152,1.0,0.153461,6.363189,18.303925,C,9.314079,15.231789,17.142857,11.784549,F,1.0,1.614988,B,D,2.230940,7.292418,8.571429,E,3.000000,7.528326,8.861647,0.649820,1.299638,1.707317,0.866426,9.551836,3.321300,0.095678,0.905342,A,0.442252,5.814018,3.517720,0.462019,7.436824,5.454545,8.877414,1.191337,19.470199,8.389237,2.757375,4.374296,1.574039,0.007294,12.579184,E,2.382692,3.930922,B,0.433213,O,G,15.634907,2.857144,1.951220,6.592012,5.909091,-6.297423e-07,1.059603,0.803572,8.000000,1.989780,0.035754,AU,1.804126,3.113719,2.024285,0.0,0.636365,2.857144
1,1,1.299827,9.382921,C,4.240272,9.191265,2.421430,2.785484,2.301630,9.282659,1.312910,15.285161,6.507647,3.643881,11.636386,2.191167,5.285996,4.119355,1.120362,0.190338,18.291107,6.763110,GUV,8.094794e-01,C,3.056144,1.872672,3.294889,6.836006,8.154094,C,A,1.654727,1.823793,3.615077,7.579133,14.579479,0.527667,0.0,1.200348,14.305766,6.943040,12.797649,2.283022,11.004618,8.362798,2.449959,E,11.955125,7.552990,1.379210,6.198177,G,15.523897,1.129469,1.121361,DY,4.138063,8.848918,10.373129,1.743861,15.026338,2.0,2.544736,6.887613,16.588786,A,9.067649,17.951084,7.962374,12.053353,F,2.0,2.533144,B,D,2.554388,7.407473,13.437474,D,2.093859,7.277655,3.430691,1.810109,1.447179,2.356586,1.292933,9.848004,1.579004,2.678584,0.893307,B,0.475052,5.959666,3.949775,0.532681,6.341732,6.859188,8.303967,1.336207,11.101371,6.989371,2.468097,5.275296,2.679005,1.505335,12.632661,B,1.825361,4.247858,A,3.225650,U,G,10.308044,12.033299,2.374528,10.595357,7.386855,3.138774e+00,0.930169,2.428448,7.188800,2.886836,0.598896,AF,1.781780,3.481543,1.957825,0.0,1.350033,1.354401
2,1,0.943877,5.310079,C,4.410969,5.326159,3.979592,3.928571,0.019645,12.666667,0.765864,14.756098,6.384670,2.505589,9.603542,1.984127,5.882353,3.170847,0.244541,0.144258,17.952332,5.245035,FQ,-2.785053e-07,E,0.113997,2.244897,5.306122,0.836005,7.499999,C,A,1.454082,1.734693,4.043864,7.959184,12.730517,0.259740,0.0,7.378964,13.077201,6.173469,12.346939,2.926830,8.897561,5.343819,0.126035,C,12.711328,6.836734,0.604504,9.637627,F,15.102041,0.085573,0.765305,AS,4.030613,4.277456,9.105481,2.151361,16.075602,1.0,0.123643,5.517949,16.377205,A,8.367347,11.040463,5.882353,8.460654,B,3.0,2.413618,B,B,1.963971,5.918368,11.764705,E,3.333334,10.194433,8.266200,1.530611,1.530613,2.429906,1.071429,8.447465,3.367346,0.111388,0.811447,G,0.271480,5.156559,4.214944,0.309657,5.663265,5.974026,11.588858,0.841837,15.491329,5.879353,3.292788,5.924457,1.668401,0.008275,11.670572,C,1.375753,1.184211,B,3.367348,S,G,11.205561,12.941177,3.129253,3.478911,6.233767,-2.792745e-07,2.138728,2.238806,9.333333,2.477596,0.013452,AE,1.773709,3.922193,1.120468,2.0,0.883118,1.176472
3,1,0.797415,8.304757,C,4.225930,11.627438,2.097700,1.987549,0.171947,8.965516,6.542669,16.347483,9.646653,3.903302,14.094723,1.945044,5.517242,3.610789,1.224114,0.231630,18.376407,7.517125,ACUE,-4.805344e-07,D,0.148843,1.308269,2.303640,8.926662,8.874521,C,B,1

In [207]:
#checking cardinality in categorical features

In [208]:
for feature in categorical_features:
    print(feature,len(train_data[feature].unique()))

v3 3
v22 1458
v24 5
v30 7
v31 3
v47 8
v52 12
v56 66
v66 3
v71 4
v74 3
v75 3
v79 16
v91 7
v107 7
v110 3
v112 22
v113 33
v125 89


In [209]:
train_data = encoding_categorical_data(categorical_features,train_data,'target')

In [210]:
catboost_training_data.head()

,target,v1,v2,v3,v4,v5,v6,v7,v8,v9,v10,v11,v12,v13,v14,v15,v16,v17,v18,v19,v20,v21,v22,v23,v24,v25,v26,v27,v28,v29,v30,v31,v32,v33,v34,v35,v36,v37,v38,v39,v40,v41,v42,v43,v44,v45,v46,v47,v48,v49,v50,v51,v52,v53,v54,v55,v56,v57,v58,v59,v60,v61,v62,v63,v64,v65,v66,v67,v68,v69,v70,v71,v72,v73,v74,v75,v76,v77,v78,v79,v80,v81,v82,v83,v84,v85,v86,v87,v88,v89,v90,v91,v92,v93,v94,v95,v96,v97,v98,v99,v100,v101,v102,v103,v104,v105,v106,v107,v108,v109,v110,v111,v112,v113,v114,v115,v116,v117,v118,v119,v120,v121,v122,v123,v124,v125,v126,v127,v128,v129,v130,v131
0,1,1.335739,8.727474,C,3.921026,7.915266,2.599278,3.176895,0.012941,9.999999,0.503281,16.434108,6.085711,2.866830,11.636387,1.355013,8.571429,3.670350,0.106720,0.148883,18.869283,7.730923,XDX,-1.716131e-08,C,0.139412,1.720818,3.393503,0.590122,8.880867,C,A,1.083033,1.010829,7.270147,8.375452,11.326592,0.454546,0,4.012088,7.711453,7.653429,12.707581,2.015505,10.498338,9.848672,0.113561,C,12.171733,8.086643,0.899420,7.277792,G,16.747968,0.037096,1.299638,DI,3.971118,0.529802,10.890984,1.588448,15.858152,1,0.153461,6.363189,18.303925,C,9.314079,15.231789,17.142857,11.784549,F,1,1.614988,B,D,2.230940,7.292418,8.571429,E,3.000000,7.528326,8.861647,0.649820,1.299638,1.707317,0.866426,9.551836,3.321300,0.095678,0.905342,A,0.442252,5.814018,3.517720,0.462019,7.436824,5.454545,8.877414,1.191337,19.470199,8.389237,2.757375,4.374296,1.574039,0.007294,12.579184,E,2.382692,3.930922,B,0.433213,O,NaN,15.634907,2.857144,1.951220,6.592012,5.909091,-6.297423e-07,1.059603,0.803572,8.000000,1.989780,0.035754,AU,1.804126,3.113719,2.024285,0,0.636365,2.857144
1,1,NaN,NaN,C,NaN,9.191265,NaN,NaN,2.301630,NaN,1.312910,NaN,6.507647,NaN,11.636386,NaN,NaN,NaN,NaN,NaN,NaN,6.763110,GUV,NaN,C,3.056144,NaN,NaN,NaN,NaN,C,A,NaN,NaN,3.615077,NaN,14.579479,NaN,0,NaN,14.305766,NaN,NaN,NaN,NaN,NaN,2.449959,E,NaN,NaN,1.379210,NaN,G,NaN,1.129469,NaN,DY,NaN,NaN,NaN,NaN,NaN,2,2.544736,NaN,NaN,A,NaN,NaN,NaN,12.053353,F,2,NaN,B,D,NaN,NaN,NaN,D,NaN,7.277655,3.430691,NaN,NaN,NaN,NaN,9.848004,NaN,2.678584,NaN,B,NaN,NaN,NaN,NaN,NaN,NaN,8.303967,NaN,NaN,NaN,NaN,NaN,NaN,1.505335,NaN,B,1.825361,4.247858,A,NaN,U,G,10.308044,NaN,NaN,10.595357,NaN,NaN,NaN,NaN,NaN,NaN,0.598896,AF,NaN,NaN,1.957825,0,NaN,NaN
2,1,0.943877,5.310079,C,4.410969,5.326159,3.979592,3.928571,0.019645,12.666667,0.765864,14.756098,6.384670,2.505589,9.603542,1.984127,5.882353,3.170847,0.244541,0.144258,17.952332,5.245035,FQ,-2.785053e-07,E,0.113997,2.244897,5.306122,0.836005,7.499999,NaN,A,1.454082,1.734693,4.043864,7.959184,12.730517,0.259740,0,7.378964,13.077201,6.173469,12.346939,2.926830,8.897561,5.343819,0.126035,C,12.711328,6.836734,0.604504,9.637627,F,15.102041,0.085573,0.765305,AS,4.030613,4.277456,9.105481,2.151361,16.075602,1,0.123643,5.517949,16.377205,A,8.367347,11.040463,5.882353,8.460654,B,3,2.413618,B,B,1.963971,5.918368,11.764705,E,3.333334,10.194433,8.266200,1.530611,1.530613,2.429906,1.071429,8.447465,3.367346,0.111388,0.811447,G,0.271480,5.156559,4.214944,0.309657,5.663265,5.974026,11.588858,0.841837,15.491329,5.879353,3.292788,5.924457,1.668401,0.008275,11.670572,C,1.375753,1.184211,B,3.367348,S,NaN,11.205561,12.941177,3.129253,3.478911,6.233767,-2.792745e-07,2.138728,2.238806,9.333333,2.477596,0.013452,AE,1.773709,3.922193,1.120468,2,0.883118,1.176472
3,1,0.797415,8.304757,C,4.225930,11.627438,2.097700,1.987549,0.171947,8.965516,6.542669,16.347483,9.646653,3.903302,14.094723,1.945044,5.517242,3.610789,1.224114,0.231630,18.376407,7.517125,ACUE,-4.805344e-07,D,0.148843,1.308269,2.303640,8.926662,8.874521,C,B,1.587644,1.666667,8.703550,8.898468,11.302795,0.433735,0,0.287322,11.523045,7.931035,12.935823,1.470878,12.708574,9.670823,0.108387,C,12.194855,8.591954,3.329176,4.780357,H,16.621695,0.139721,1.178161,BW,3.965517,1.732102,11.777912,1.229246,15.927390,1,0.140260,6.292979,17.011645,A,9.703065,18.568129,9.425288,13.594728,F,2,2.272541,B,D,2.188198,8.213602,13.448277,B,1.947261,4.797873,13.315819,1.681034,1.379310,1.587045,1.242817,10.747144,1.40

In [211]:
#x_train,x_test,y_train,y_test=train_test_split(train_data.drop(columns='target'),train_data['target'],test_size=0.3,random_state=23)

In [212]:
"""
for feature in train_data.columns:
    if train_data[feature].isnull().sum()>0:
        print(feature)
        print(train_data[feature].isnull().sum())
"""

'\nfor feature in train_data.columns:\n    if train_data[feature].isnull().sum()>0:\n        print(feature)\n        print(train_data[feature].isnull().sum())\n'

In [213]:
# print('x_train',x_train.shape)
# print('y_train',y_train.shape)
# print('x_test',x_test.shape)
# print('y_test',y_test.shape)

In [214]:
for feature in catboost_training_data.columns:
    if catboost_training_data[feature].isnull().sum()>0:
        print(feature)

v1
v2
v3
v4
v5
v6
v7
v8
v9
v11
v13
v15
v16
v17
v18
v19
v20
v21
v22
v23
v25
v26
v27
v28
v29
v30
v31
v32
v33
v35
v36
v37
v39
v41
v42
v43
v44
v45
v46
v48
v49
v51
v53
v54
v55
v56
v57
v58
v59
v60
v61
v63
v64
v65
v67
v68
v69
v70
v73
v76
v77
v78
v80
v81
v82
v83
v84
v85
v86
v87
v88
v89
v90
v92
v93
v94
v95
v96
v97
v98
v99
v100
v101
v102
v103
v104
v105
v106
v108
v109
v111
v112
v113
v115
v116
v117
v118
v119
v120
v121
v122
v123
v124
v126
v127
v128
v130
v131


In [215]:
global models_results
models_results = defaultdict(lambda :[])

In [216]:
catboost_training_data=catboost_training_data.dropna()

In [217]:
x1=catboost_training_data.drop(columns='target')
y1=catboost_training_data['target']
skf = StratifiedKFold(n_splits=5,shuffle=True,random_state=1)
x2 = np.array(x1)
y2= np.array(y1) 
model=CatBoostClassifier(iterations=100,depth=8,learning_rate=0.1,
                        cat_features=categorical_features,loss_function='Logloss',
                        custom_metric=['AUC'],random_seed=42)
global models_results
models_results = defaultdict(lambda :[])
for train_index,test_index in skf.split(x1,y1):
    x_train_fold ,x_test_fold = x1.iloc[train_index],x1.iloc[test_index]
    y_train_fold,y_test_fold = y1.iloc[train_index],y1.iloc[test_index]
    model.fit(x_train_fold,y_train_fold)
    probabilities = model.predict_proba(x_test_fold)[:,1]
    y_pred = model.predict(x_test_fold)
    accuracy = accuracy_score(y_test_fold,y_pred)
    precision_score1= precision_score(y_pred,y_test_fold)
    recall_score1 = recall_score(y_pred,y_test_fold)
    logloss = log_loss(y_test_fold,probabilities)
    models_results['catboost'+'_'+'accuracy'].append(accuracy)
    models_results['catboost'+'_'+'precision_score'].append(precision_score1)
    models_results['catboost'+'_'+'recall_score'].append(recall_score1)
    models_results['catboost'+'_'+'logloss'].append(logloss)

0:	learn: 0.6429273	total: 266ms	remaining: 26.3s
1:	learn: 0.6280726	total: 332ms	remaining: 16.3s
2:	learn: 0.5851415	total: 614ms	remaining: 19.9s
3:	learn: 0.5486636	total: 910ms	remaining: 21.8s
4:	learn: 0.5124876	total: 1.21s	remaining: 23s
5:	learn: 0.4821572	total: 1.57s	remaining: 24.6s
6:	learn: 0.4736329	total: 1.69s	remaining: 22.5s
7:	learn: 0.4631097	total: 1.75s	remaining: 20.1s
8:	learn: 0.4339708	total: 2.06s	remaining: 20.8s
9:	learn: 0.4123280	total: 2.34s	remaining: 21s
10:	learn: 0.3897416	total: 2.56s	remaining: 20.7s
11:	learn: 0.3660423	total: 2.77s	remaining: 20.3s
12:	learn: 0.3505688	total: 2.96s	remaining: 19.8s
13:	learn: 0.3373724	total: 3.15s	remaining: 19.3s
14:	learn: 0.3221673	total: 3.33s	remaining: 18.9s
15:	learn: 0.3033246	total: 3.76s	remaining: 19.7s
16:	learn: 0.2951220	total: 4.05s	remaining: 19.8s
17:	learn: 0.2839249	total: 4.36s	remaining: 19.9s
18:	learn: 0.2680520	total: 4.64s	remaining: 19.8s
19:	learn: 0.2503637	total: 4.92s	remaining: 

In [218]:
models_results

defaultdict(<function __main__.<lambda>()>,
            {'catboost_accuracy': [0.7301587301587301,
              0.746031746031746,
              0.6290322580645161,
              0.7258064516129032,
              0.8064516129032258],
             'catboost_precision_score': [0.9555555555555556,
              0.9333333333333333,
              0.8,
              0.9545454545454546,
              1.0],
             'catboost_recall_score': [0.7413793103448276,
              0.7636363636363637,
              0.72,
              0.7368421052631579,
              0.7857142857142857],
             'catboost_logloss': [0.5997715000376819,
              0.6104649770103772,
              0.6810905199589162,
              0.5937718719668437,
              0.5665520165132193]})

In [219]:
print('accuracy',np.mean(models_results['catboost_accuracy']))
print('precision',np.mean(models_results['catboost_precision_score']))
print('recall',np.mean(models_results['catboost_recall_score']))
print('logloss',np.mean(models_results['catboost_logloss']))

accuracy 0.7274961597542242
precision 0.9286868686868687
recall 0.7495144129917269
logloss 0.6103301770974077


In [220]:
x=train_data.drop(columns='target')
y=train_data['target']

In [221]:
print(x.shape)
print(y.shape)

(2000, 131)
(2000,)


Experiment2

In [222]:
#smote = SMOTE(sampling_strategy='auto', random_state=42)
smote = SMOTE(sampling_strategy='minority', random_state=42)
x_resampled, y_resampled = smote.fit_resample(x, y.ravel())

In [223]:
x_resampled['target']=y_resampled

C:\Users\Dell\AppData\Local\Temp\ipykernel_21972\2966117113.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  x_resampled['target']=y_resampled


In [224]:
x_resampled['target'].value_counts()

target
1    1522
0    1522
Name: count, dtype: int64

In [225]:
x_resampled.drop(columns='target',inplace=True)

In [226]:
skf = StratifiedKFold(n_splits=5,shuffle=True,random_state=1)

In [228]:
logistic=LogisticRegression()
scaler=MinMaxScaler()
x_scaled=pd.DataFrame(scaler.fit_transform(x_resampled),columns=x_resampled.columns)
x_scaled1 = np.array(x_scaled)
y_scaled1 = np.array(y_resampled)
for train_index,test_index in skf.split(x_scaled,y_resampled):
    x_train_fold ,x_test_fold = x_scaled1[train_index],x_scaled1[test_index]
    y_train_fold,y_test_fold = y_scaled1[train_index],y_scaled1[test_index]
    logistic.fit(x_train_fold,y_train_fold)
    probabilities = logistic.predict_proba(x_test_fold)[:,1]
    predictions = logistic.predict(x_test_fold)
    accuracy = accuracy_score(predictions,y_test_fold)
    precision_score1 = precision_score(predictions,y_test_fold)
    recall_score1 = recall_score(predictions,y_test_fold)
    logloss = log_loss(y_test_fold,probabilities)
    models_results['Logistic Regression'+"_"+'accuracy'].append(accuracy)
    models_results['Logistic Regression'+"_"+'precision_score'].append(precision_score1)
    models_results['Logistic Regression'+"_"+'recall_score'].append(recall_score1)
    models_results['Logistic Regression'+"_"+'logloss'].append(logloss)

In [229]:
def xgb_mlp_model(x_train,y_train,x_test,y_test,models_results):
    xgb_train = xgb.DMatrix(x_train,y_train,enable_categorical=True)
    xgb_test = xgb.DMatrix(x_test,y_test,enable_categorical = True)
    params = {
        'objective':'binary:logistic',
        'max_depth':3,
        'learning_rate':0.1,
    }
    model = xgb.train(params=params,dtrain=xgb_train,num_boost_round=50)
    preds_proba = model.predict(xgb_test)
    logloss=log_loss(y_test,preds_proba)
    preds = (preds_proba > 0.5).astype(int) 
    accuracy = accuracy_score(y_test,preds)
    precision_score1 = precision_score(preds,y_test)
    recall_score1 = recall_score(preds,y_test)
    models_results['xgb'+"_"+'accuracy'].append(accuracy)
    models_results['xgb'+"_"+'precision_score'].append(precision_score1)
    models_results['xgb'+'_'+'recall_score'].append(recall_score1)
    models_results['xgb'+'_'+'logloss'].append(logloss)
    #model_scores.append(("xg boost",accuracy,precision_score1,recall_score1,logloss))

    mlp_model = Sequential()
    mlp_model.add(Dense(32,activation='relu',input_dim = x_train.shape[1]))
    mlp_model.add(Dense(16,activation='relu'))
    mlp_model.add(Dense(8,activation='relu'))
    mlp_model.add(Dense(1,activation='sigmoid'))
    mlp_model.compile(optimizer='Adam',loss='binary_crossentropy',metrics=['accuracy'])
    history = mlp_model.fit(x_train,y_train,batch_size=500,epochs=100,verbose=0,validation_split=0.2)
    y_pred_probs = mlp_model.predict(x_test).flatten()
    y_pred = mlp_model.predict(x_test)
    y_pred = (y_pred > 0.5).astype(int) 
    accuracy = accuracy_score(y_test,y_pred)
    precision_score1= precision_score(y_pred,y_test)
    recall_score1 = recall_score(y_pred,y_test)
    logloss = log_loss(y_test,y_pred_probs)
    #model_scores.append(("MLP",accuracy,precision_score1,recall_score1,logloss))

    #models_df = pd.DataFrame(model_scores,columns=['model','accuracy','precision','recall','logloss'])
    models_results['MLP'+"_"+'accuracy'].append(accuracy)
    models_results['MLP'+"_"+'precision_score'].append(precision_score1)
    models_results['MLP'+'_'+'recall_score'].append(recall_score1)
    models_results['MLP'+'_'+'logloss'].append(logloss)
    
    return models_results

In [230]:
def cross_validation(x_resampled,y_resampled,models_results):
    # skf = StratifiedKFold(n_splits=5,shuffle=True,random_state=1)
    models=[("Decision tree",DecisionTreeClassifier()),
            ("Random forest",RandomForestClassifier()),("Naive bayes",GaussianNB()),('adaboost',AdaBoostClassifier()),('gradientboosting classifier',GradientBoostingClassifier())]
    #models_results = defaultdict(lambda :[])
    x_resampled1 = np.array(x_resampled)
    y_resampled1 = np.array(y_resampled)
    for c,m in models:
        for train_index,test_index in skf.split(x_resampled,y_resampled):
            x_train_fold ,x_test_fold = x_resampled1[train_index],x_resampled1[test_index]
            y_train_fold,y_test_fold = y_resampled1[train_index],y_resampled1[test_index]
            m.fit(x_train_fold,y_train_fold)
            probabilities = m.predict_proba(x_test_fold)[:,1]
            predictions = m.predict(x_test_fold)
            accuracy = accuracy_score(predictions,y_test_fold)
            precision_score1 = precision_score(predictions,y_test_fold)
            recall_score1 = recall_score(predictions,y_test_fold)
            logloss = log_loss(y_test_fold,probabilities)
            models_results[c+"_"+'accuracy'].append(accuracy)
            models_results[c+"_"+'precision_score'].append(precision_score1)
            models_results[c+"_"+'recall_score'].append(recall_score1)
            models_results[c+"_"+'logloss'].append(logloss)
            models_results = xgb_mlp_model(x_train_fold,y_train_fold,x_test_fold,y_test_fold,models_results)
        
    return models_results

In [231]:
models_results2 = cross_validation(x_resampled,y_resampled,models_results)
#print(models_results)

C:\Users\Dell\AppData\Roaming\Python\Python311\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step


C:\Users\Dell\AppData\Roaming\Python\Python311\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step


C:\Users\Dell\AppData\Roaming\Python\Python311\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


...

20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step


C:\Users\Dell\AppData\Roaming\Python\Python311\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step


C:\Users\Dell\AppData\Roaming\Python\Python311\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step


Experiment 3

In [232]:
new_model_results = {}
cross_validation_results = []
for key,value in models_results2.items():
    m = key.split('_')[0]
    metric = key.split('_')[1]
    if key.split('_')[1] in ('accuracy','precision','recall'):
        new_model_results[key] = np.mean(value)
    if key.split('_')[1] in ('logloss'):
        new_model_results[key] = np.mean(value)
    cross_validation_results.append((m,metric,new_model_results[key]))

In [233]:
#cross_validation_results

In [234]:
cross_validations_df = pd.DataFrame(cross_validation_results,columns=['model','metric','value'])
cross_validations_df.to_csv('cross_validations_results.csv',index=False)

In [235]:
cross_validations_df

,model,metric,value
0,catboost,accuracy,0.727496
1,catboost,precision,0.928687
2,catboost,recall,0.749514
3,catboost,logloss,0.610330
4,Logistic Regression,accuracy,0.923460
5,Logistic Regression,precision,0.913913
6,Logistic Regression,recall,0.931853
7,Logistic Regression,logloss,0.176328
8,Decision tree,accuracy,0.921485
9,Decision tree,precision,0.914586
